# **DeepSpell**
---




### Now a days we are using lots of digital documentation and formal emails. And we are creating it manually which can cause spelling mistakes. Misspelling can be hindrance in understanding of documents. 
### If we will talk about one word, there can be four possibility for errors :

### *   when you write extra letter in a word
### *   when you transpose by mistake
### *   when you type wrong letter 
### *   when you forget to write a letter


### And this can be corrected in two steps:



### *   Using cartesian product by creating probability of known words(dictionary words) and replace the misspelled word.
### *   From the set of words, identifying the best possible words that is correct.


## About Code :

### Here they are using an algorithm for Spelling correction in English texts using machine learning. They are using artificial data set by manually adding errors in correct sentences. They are using data set of 286MB text file news that is a part of one billion words and train with character by character and sequence-to-sequence model with  Long Short Term Memory networks(a special kind of RNN) layers. Which is helping in conversion of sentences with error to correct text. 
### For cleaning the text we are using preprocessing pipeline, parses into lines, putting input as artificial spelling errors. And removed the space,dash,parenthesis etc.


In [0]:
def clean_text(text):
    """Clean the text - remove unwanted chars, fold punctuation etc."""
    result = NORMALIZE_WHITESPACE_REGEX.sub(' ', text.strip())
    result = RE_DASH_FILTER.sub('-', result)
    result = RE_APOSTROPHE_FILTER.sub("'", result)
    result = RE_LEFT_PARENTH_FILTER.sub("(", result)
    result = RE_RIGHT_PARENTH_FILTER.sub(")", result)
    result = RE_BASIC_CLEANER.sub('', result)
    return result

def preprocesses_data_clean():
    """Pre-process the data - step 1 - cleanup"""
    with open(NEWS_FILE_NAME_CLEAN, "wb") as clean_data:
        for line in open(NEWS_FILE_NAME):
            decoded_line = line.decode('utf-8')
            cleaned_line = clean_text(decoded_line)
            encoded_line = cleaned_line.encode("utf-8")
            clean_data.write(encoded_line + b"\n")

###Then breaking every lines into chunks. And each chunks is smaller than 40 words. Resulting pipeline is a set of question/answer pairs where question/answer are strings. Here question contain spelling mistakes and answer will be the string with approximatly correct strings with deep learning.

In [0]:
def preprocesses_split_lines():
    LOGGER.info("Reading filtered data:")
    answers = set()
    with open(NEWS_FILE_NAME_SPLIT, "wb") as output_file:
        for _line in open(NEWS_FILE_NAME_FILTERED):
            line = _line.decode('utf-8')
            while len(line) > MIN_INPUT_LEN:
                if len(line) <= CONFIG.max_input_len:
                    answer = line
                    line = ""
                else:
                    space_location = line.rfind(" ", MIN_INPUT_LEN, CONFIG.max_input_len - 1)
                    if space_location > -1:
                        answer = line[:space_location]
                        line = line[len(answer) + 1:]
                    else:
                        space_location = line.rfind(" ") # no limits this time
                        if space_location == -1:
                            break # we are done with this line
                        else:
                            line = line[space_location + 1:]
                            continue
                answers.add(answer)
                output_file.write(answer.encode('utf-8') + b"\n")

###They are using 90% of data as training dataset and 10% is learning dataset. Here they are using small preprocessed  and kept most of the data most popular characters. 
###They are using noise data to process by adding artificial error dataset.

In [0]:
def add_noise_to_string(a_string, amount_of_noise):
    """Add some artificial spelling mistakes to the string"""
    if rand() < amount_of_noise * len(a_string):
        # Replace a character with a random character
        random_char_position = random_randint(len(a_string))
        a_string = a_string[:random_char_position] + random_choice(CHARS[:-1]) + a_string[random_char_position + 1:]
    if rand() < amount_of_noise * len(a_string):
        # Delete a character
        random_char_position = random_randint(len(a_string))
        a_string = a_string[:random_char_position] + a_string[random_char_position + 1:]
    if len(a_string) < CONFIG.max_input_len and rand() < amount_of_noise * len(a_string):
        # Add a random character
        random_char_position = random_randint(len(a_string))
        a_string = a_string[:random_char_position] + random_choice(CHARS[:-1]) + a_string[random_char_position:]
    if rand() < amount_of_noise * len(a_string):
        # Transpose 2 characters
        random_char_position = random_randint(len(a_string) - 1)
        a_string = (a_string[:random_char_position] + a_string[random_char_position + 1] + a_string[random_char_position] +
                    a_string[random_char_position + 2:])
    return a_string

In [0]:
def generate_question(answer):
    """Generate a question by adding noise"""
    question = add_noise_to_string(answer, AMOUNT_OF_NOISE)
    # Add padding:
    question += PADDING * (CONFIG.max_input_len - len(question))
    answer += PADDING * (CONFIG.max_input_len - len(answer))
    return question, answer

###They are using vector based approach. The model learns the association of other words with each other and the pattern in which it occures. From this steps, each word would be converted to a vector and same related words will have vectors which are closer in distance. 

In [0]:
def _vectorize(questions, answers, ctable):
    """Vectorize the data as numpy arrays"""
    len_of_questions = len(questions)
    X = np_zeros((len_of_questions, CONFIG.max_input_len, ctable.size), dtype=np.bool)
    for i in xrange(len(questions)):
        sentence = questions.pop()
        for j, c in enumerate(sentence):
            try:
                X[i, j, ctable.char_indices[c]] = 1
            except KeyError:
                pass # Padding
    y = np_zeros((len_of_questions, CONFIG.max_input_len, ctable.size), dtype=np.bool)
    for i in xrange(len(answers)):
        sentence = answers.pop()
        for j, c in enumerate(sentence):
            try:
                y[i, j, ctable.char_indices[c]] = 1
            except KeyError:
                pass # Padding
    return X, y

In [0]:
def vectorize(questions, answers, chars=None):
    """Vectorize the questions and expected answers"""
    print('Vectorization...')
    chars = chars or CHARS
    ctable = CharacterTable(chars)
    X, y = _vectorize(questions, answers, ctable)
    # Explicitly set apart 10% for validation data that we never train over
    split_at = int(len(X) - len(X) / 10)
    (X_train, X_val) = (slice_X(X, 0, split_at), slice_X(X, split_at))
    (y_train, y_val) = (y[:split_at], y[split_at:])

    print(X_train.shape)
    print(y_train.shape)

    return X_train, X_val, y_train, y_val, CONFIG.max_input_len, ctable

### When we create the set, misspelled word will replace by corrected word and pass the set to word2vector deep learning engine. Then they identify the vector distance between context words and word that have to replace, wchich is closest to  the vector of nearby words. And it is selected as the final output.

## Future Improvement :

### *   Dataset are artificial, we can use real life typing mistake dataset
### *   We can use current version of keras
### *   We can use TenserFlow API
### *   We can use more larger dataset.

---
#Thank You :)
---